<a href="https://colab.research.google.com/github/ducanh2505/scrape_data/blob/main/utilize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages

In [2]:
!pip install requests-html

     |████████████████████████████████| 153kB 5.1MB/s 
     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 143kB 6.3MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 
  Created wheel for parse: filename=parse-1.18.0-cp36-none-any.whl size=24133 sha256=e942177046652842eb346d657248805b11e55f9403514964297f46b58db43ec2
  Stored in directory: /root/.cache/pip/wheels/2a/53/09/869ca5781ede342254ffac09ca99461b008c3e5f8dd079b0c0
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=6b290e98b655049590d653ffb0d3fa37ecbd0c063dc068afe3bf0b0b9761f25a
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built parse fake-useragent
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have fol

### import packages and define some variables

In [3]:

from requests_html import HTMLSession
import requests
import json
import re
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
session = HTMLSession()
HOME = 'https://gearvn.com'


### get_cat_menu()


In [41]:
r = session.get(home)
len(np.unique([t.find('a',first=True).attrs['href'] for t in r.html.find('li.cat-menu-item')]))


15

In [42]:
def get_cat_menu():
  r = session.get(home)
  return np.unique([t.find('a',first=True).attrs['href'] for t in r.html.find('li.cat-menu-item')]).tolist()
cat_menu = get_cat_menu()
cat_menu

['/collections/ban-phim-co-gaming',
 '/collections/gaming-headphones',
 '/collections/gaming-mouse',
 '/collections/gaming-pc-gearvn',
 '/collections/ghe-choi-game',
 '/collections/laptop',
 '/collections/linh-kien-may-tinh',
 '/collections/man-hinh',
 '/collections/may-choi-game',
 '/collections/pc-van-phong',
 '/collections/pc-workstation',
 '/collections/phu-kien',
 '/collections/thiet-bi-mang',
 '/collections/thiet-bi-tai-nghe-loa-audio-chuyen-nghiep',
 '/collections/thiet-bi-van-phong']

### def get_from_page()
get all links from page

In [43]:
url = 'https://gearvn.com/collections/laptop'
def get_from_page(url,session=session):  
  r = session.get(url)
  product_row = r.html.find('.product-row')
  return [home + link.find('a[href]',first=True).attrs['href']\
                            for link in product_row]
a = get_from_page(url)

In [ ]:
len(a),a

(20,
 ['https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa',
  'https://gearvn.com/products/laptop-asus-vivobook-d509da-ej116t-new',
  'https://gearvn.com/products/laptop-asus-d409da-ek499t',
  'https://gearvn.com/products/laptop-hp-15s-fq1107tu-193q3pa',
  'https://gearvn.com/products/laptop-acer-aspire-3-a315-56-37dv',
  'https://gearvn.com/products/laptop-asus-x509ja-ej427t',
  'https://gearvn.com/products/laptop-asus-a412fa-ek1187t',
  'https://gearvn.com/products/laptop-asus-experbook-p1410cja-ek354t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-54-39ku',
  'https://gearvn.com/products/laptop-asus-vivobook-flip-tm420ia-ec155t',
  'https://gearvn.com/products/laptop-asus-vivobook-a515ea-bq489t',
  'https://gearvn.com/products/laptop-asus-x509ja-ej021t',
  'https://gearvn.com/products/laptop-asus-x509jp-ej012t',
  'https://gearvn.com/products/laptop-acer-aspire-5-a514-53-38gf',
  'https://gearvn.com/products/laptop-asus-vivobook-x409ja-ek014t-silver',
  'htt

### get_from_site()
scrawl all link from the site

In [44]:
url = 'https://gearvn.com/collections/laptop'
def get_product_from_site(url,par=None,session= session):
  finish = False
  page = 1
  all_links = []
  while not finish:
    t = get_from_page(url+'?page={}'.format(page),session)
    if (len(t) > 0 ):
      page += 1
      all_links.extend(t)
      # if par :
      #   progess = {}
      #   progess['len'] = len(t)
      #   par.update()
    else: 
      finish = True
  return all_links



In [45]:
all_links =get_product_from_site(url)
len(all_links)


103

### get_all_product()


In [46]:
def get_all_product(cat_menu=cat_menu):
  product_links = []
  par = tqdm(cat_menu,total =len(cat_menu))
  progess = {}
  for category in par:
    t = get_product_from_site(HOME+category)    
    product_links.extend(t)
    progess['len'] = len(t)
    progess['total'] = len(product_links)
    par.set_postfix(progess)
    par.update()
  return product_links
product_links = get_all_product()

In [47]:
with open("product_links.txt",'w') as f:
  f.writelines([product+'\n' for product in product_links])

### get_from_product()
scrawl from a product

In [ ]:
product = all_links[0]
product

'https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa'

In [35]:
# r = session.get('https://gearvn.com/products/laptop-gaming-asus-rog-zephyrus-m-gu502gv-az079t')
# r = session.get('https://gearvn.com/products/laptop-hp-envy-13-ba1028tu-2k0b2pa')
r = session.get('https://gearvn.com/products/laptop-acer-aspire-5-a514-54-51rb?fbclid=IwAR1wiKV3U5i6_nA9MOP0KGvO9l4aZMiN4ihqFvplA-apEeaJF9Jg4h4cEQY')
def get_parse_text(r,product_dict = None):
  text = r.html.find('.product_parameters')[0].find('p')[0].text.split("\n") 
  text = [t.split(':') for t in text] 
  if product_dict == None:
    product_dict = {}
  for index, param in enumerate(text):  
    if len(param) > 1: 
      if param[1] == '' or param[0] == 'Quà tặng':
        if param[0] == 'Quà tặng':
          for next_index in range(index + 1, len(text)):          
            if len(text[next_index]) == 1 and text[next_index][0] == '' and text[next_index +1][0]=='':
              break               
            param.append(text[next_index][0]) 
        else:
          for next_index in range(index + 1, len(text)):
            if len(text[next_index]) == 1 and text[next_index][0] != '':
              param.append(text[next_index][0]) 
              break
      title = re.sub('[^\w ,]+', '', param[0])
      arg = re.sub('[^\w\d ,]+', '',", ".join(param[1:]))
     
      product_dict[title] = arg
      

  return product_dict 

get_parse_text(r,{'a':1})


{'Bảo hành': ' 12 Tháng',
 'Giá Cũ': ', 16,990,000',
 'Giá KM': '15,890,000',
 'Nhà sản xuất': ' Acer',
 'Quà tặng': ' GIÁ CÓ QUÀ TẶNG 16,390,000 VNĐ, , Tai Nghe Sony MDR  EX155AP, Túi chống shock GEARVN, Chuột không dây DareU LM115G, , Balo ACER OFFICE, Ly nước ACER',
 'Tình trạng': ' Mới 100',
 'Xuất xứ': ' Chính hãng',
 'a': 1}

In [39]:
def get_from_product(links):
  products = [] 
  bar = tqdm(links,total = len(links))
  for product in bar:
    r = session.get(product)
    product_dictionary = {}

    # get general atributes
    general_attributes = r.html.find('table>tbody',first=True).find('tr')
    for attr in general_attributes:   
      if len(attr.find('td')) > 1 :
        product_dictionary[attr.find('td')[0].text] = attr.find('td')[1].text 
                                              


    # get product parameter
    product_dictionary = get_parse_text(r,product_dictionary)

    products.append(product_dictionary)
  return products

In [40]:
links = ['https://gearvn.com/products/laptop-acer-aspire-5-a514-54-51rb?fbclid=IwAR1wiKV3U5i6_nA9MOP0KGvO9l4aZMiN4ihqFvplA-apEeaJF9Jg4h4cEQY']
get_from_product(links)

[{'Audio': 'Acer TrueHarmony technology',
  'Bluetooth': 'v5.1',
  'Bảo hành': ' 12 Tháng',
  'CPU': 'Intel Core i5-1135G7 2.4GHz up to 4.2GHz 8MB',
  'Card đồ họa': 'Intel Iris Xe Graphics',
  'Chuẩn LAN': 'Gigabit Ethernet',
  'Chuẩn WIFI': 'Intel® Wireless Wi-Fi 6 AX201 (2x2)',
  'Cổng giao tiếp': '1x USB 3.2 port with power-off charging\n2x USB 3.2 port\n1x USB Type-C port\n1x RJ-45 port\n1x HDMI® port',
  'Giá Cũ': ', 16,990,000',
  'Giá KM': '15,890,000',
  'Hệ điều hành': 'Windows 10 Home',
  'Kích thước': '328 x 233 x 17.95 (mm)',
  'Màn hình': '14" FHD (1920 x 1080) IPS, Acer ComfyView LCD',
  'Màu sắc': 'Safari Gold (Vỏ nhôm)',
  'Nhà sản xuất': ' Acer',
  'Pin': '3 Cell 48WHr',
  'Quà tặng': ' GIÁ CÓ QUÀ TẶNG 16,390,000 VNĐ, , Tai Nghe Sony MDR  EX155AP, Túi chống shock GEARVN, Chuột không dây DareU LM115G, , Balo ACER OFFICE, Ly nước ACER',
  'RAM': '8GB (4GBx2) DDR4 2666MHz\xa0 (1x SO-DIMM socket, up to 20GB SDRAM)',
  'Trọng lượng': '1.46 kg',
  'Tình trạng': ' Mới 1

### Scrape data

In [53]:
product_links = []
with open("product_links.txt",'r') as f:
  product_links = f.readlines()
product_links = [line[:-1] for line in product_links]

In [54]:

products = get_from_product(product_links)

In [55]:
import json
with open('data.json', 'w') as f:
    json.dump(product_links, f)